In [2]:
import pandas as pd
import s3fs
import boto3
from io import StringIO # python3; python2: BytesIO 
import numpy as np
import re
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer

In [3]:
all_features = ["Language", "tweet_timestamp", "enaged_with_user_id", "engaging_user_id", "engagee_follows_engager",\
                "reply_timestamp"]

train = pd.read_csv("s3://recsys-challenge-2020/training.tsv", encoding="utf-8",
                    names = all_features, usecols= [7, 8, 9, 14, 19, 20], sep="\x01")

train_sorted = train.sort_values(by=['tweet_timestamp']).reset_index()
n_head = 90
n_tail = 10
train_set = train_sorted.head(int(len(train_sorted)*(n_head/100)))
val_set = train_sorted.tail(int(len(train_sorted)*(n_tail/100))).reset_index()

,Language,tweet_timestamp,enaged_with_user_id,engaging_user_id,engagee_follows_engager,reply_timestamp
0,22C448FF81263D4BAF2A176145EE9EAD,1581258144,D7EFC6D19CFB09B9A1CE4C4B0E0091F6,0000006C3074607050F1339DDCB890BB,False,NaN
1,B9175601E87101A984A50F8A62A1C374,1581018910,1F30A85E612A24B426013978ADB7AE46,000005C520010F8917EEAB6F5B6EC1C4,True,NaN
2,22C448FF81263D4BAF2A176145EE9EAD,1581244866,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,NaN
3,22C448FF81263D4BAF2A176145EE9EAD,1581250124,BEB3C337B266E2C4769065AD64724329,000006829BEADA9EEA695CF0C334B426,False,NaN
4,22C448FF81263D4BAF2A176145EE9EAD,1580958805,9E1FD934C376903895DEEBCC0F6AB920,000006829BEADA9EEA695CF0C334B426,False,NaN
